# 4.6 part 2 Combining & Exporting Data.ipynb — TOC
	1.	Setup (imports + paths)
	2.	Task: Combine orders_products_combined + products
	3.	Join issue & resolution (why row count changed)
	4.	Data cleaning on products (dedupe)
	5.	Merge (left join) on product_id
	6.	Cleanup (drop merge flags / temp cols)
	7.	Exports

# Importing

In [135]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [136]:
# Import datasets
path = r'/Users/spencer/Documents/Career Foundry/Data Immersion/4 Python Fundamentals for Data Analysts/Instacart Basket Analysis'
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'), index_col = False)
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'), index_col = False)
df_dep = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'), index_col = False)
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'orders_products_prior.csv'), index_col = False)
df_ords_prods_comb = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

# Task

In [137]:
# verify shape
df_ords_prods_comb.shape

(32434489, 10)

## Combine orders_products_combined dataframe with products data set

In [138]:
df_ords_prods_comb.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,True,196,1,0
1,2539329,1,1,2,8,NaN,True,14084,2,0
2,2539329,1,1,2,8,NaN,True,12427,3,0
3,2539329,1,1,2,8,NaN,True,26088,4,0
4,2539329,1,1,2,8,NaN,True,26405,5,0


In [139]:
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


# NOTE: Left join resulted in more values than left data set (not possible?). Went down rabbit hole with Gemini Pro for 2 hours, and concluded that df_prods HAD to have duplicates somewhere

## Data cleaning on df_prods (not assigned/done in 4.4)

In [140]:
df_prods.shape

(49672, 5)

In [141]:
print(df_prods[df_prods.duplicated()])

Empty DataFrame
Columns: [product_id, product_name, aisle_id, department_id, prices]
Index: []


In [142]:
df_dups = df_prods[df_prods.duplicated(subset=['product_id'])]
print(df_dups)

       product_id                                  product_name  aisle_id  \
6785         6800            Sprouted Quinoa Flakes Baby Cereal        92   
26505       26520  Cheese Shredded Sharp Cheddar Reduced Fat 2%        21   

       department_id  prices  
6785              18    14.0  
26505             16     2.9  


In [143]:
df_prods = df_prods.drop_duplicates(subset=['product_id'])

In [144]:
df_prods.shape

(49670, 5)

### merge (left join) on product_id

In [145]:
df_merged = df_ords_prods_comb.merge(df_prods, on = 'product_id', how = 'left', indicator = True)

In [146]:
df_merged['_merge'].value_counts()

_merge
both          32404289
left_only        30200
right_only           0
Name: count, dtype: int64

In [147]:
# expect 32434489 rows
df_merged.shape

(32434489, 15)

### Drop merge flag for data cleanliness

In [148]:
df_merged = df_merged.drop(columns = ['_merge'])

In [149]:
df_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77.0,7.0,9.0
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23.0,19.0,4.4
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23.0,19.0,4.7
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0


### Exports

In [150]:
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge.pkl'))